For interacting with the new template format while drafting proposals.

In [1]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

wb = xw.books.active
ws = wb.sheets.active
print(wb)   

<Book [J12478 MODEC - 4103 UARU FPSO - VSAT R1.xlsx]>


In [ ]:
skip_sheets = ['Config', 'Cover', 'Summary', 'Technical_Notes', 'T&C']

In [25]:
system_count = wb.sheets.count - 5 # 5 is number of default skip_sheets
remarks = {}
system_title = []
summary_sheet = wb.sheets['Summary']
offset = 20

In [5]:
summary_sheet

<Sheet [J12478 MODEC - 4103 UARU FPSO - VSAT R1.xlsx]Summary>

In [26]:
for item in range(system_count):
    remarks[summary_sheet.range(f'C{offset+item}').value] = [summary_sheet.range(f'E{offset+item}').value]

In [45]:
# Look for discount value
discount = 0
if summary_sheet.range(f'C{system_count+offset+2}').value == 'SPECIAL DISCOUNT': 
    discount = summary_sheet.range(f'D{system_count+offset+2}').value
    print(discount)

-19960.0


In [46]:
discount

-19960.0

In [32]:
cell_range = wb.app.selection
active_row = wb.app.selection.row
active_row
# ws.range(str(active_row) + ":" + str(active_row)).copy(ws.range('4:4'))
ws.range('4:4').copy(ws.range(str(active_row) + ':' + str(active_row)))
active_column = wb.app.selection.column
active_column


3

In [5]:
functions.delete_extra_empty_row(ws)

In [7]:
active_row = wb.app.selection.row
active_row
ws.range('B4').copy(ws.range('B' + str(active_row)))